# biosc - diagnostics: isochrones and $A(Li)$ model comparison

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.path import Path
import pymc as pm
import arviz as az
import bambi as bmb
import xarray as xr
import biosc
import biosc.preprocessing
import matplotlib.ticker as ticker

from pymc import HalfCauchy, Model, Normal, sample

import os
import matplotlib.cm as cm
from netCDF4 import Dataset as NetCDFFile
from scipy.stats import gaussian_kde

from biosc.preprocessing import Preprocessing
from biosc.bhm import BayesianModel

import sys
sys.path.append('/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/')

import bmp
from bmp import BayesianModelPlots

In [2]:
plt.rcParams.update({'font.size': 14, 'axes.linewidth': 1, 'axes.edgecolor': 'k'})
plt.rcParams['font.family'] = 'serif'

data_file = 'test_ALi_low_clean_only_good.csv'
file = 'output_test_UCDs_low_clean_only_good_uninf.nc'
path_data = '/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/data/test_ALi_low_clean_only_good.csv'
path_models = '/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/data/BT-Settl_all_Myr_Gaia+2MASS+PanSTARRS.csv'
L = 10
priors = {
    'Age [Myr]': {'dist': 'uniform', 'lower': 60, 'upper': 150},
    'Distance [pc]': {'dist': 'normal', 'mu': 135, 'sigma': 20}
}
ages = [0.02, 0.08, 0.12, 0.5]
colormap = 'turbo'
plot_type = 'all'

bayesian_plots = BayesianModelPlots(data_file, priors, file, path_data, path_models, L, ages, colormap)
bayesian_plots.process_idata(plot_type=plot_type)

### Pleiades data

In [3]:
data_obs_Pleiades = pd.read_csv('/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/data/Pleiades_DANCe+GDR3+2MASS+PanSTARRS1+A_Li.csv')

data_obs_Pleiades.rename(columns={'g_error': 'e_g', 'rp_error': 'e_rp', 'bp_error': 'e_bp'}, inplace=True)
data_obs_Pleiades.columns

In [4]:
distance_obs = 1 / (data_obs_Pleiades['parallax'] * 1e-3)
distance_mod_obs = 5 * np.log10(distance_obs) - 5  
data_obs_Pleiades['g_abs'] = data_obs_Pleiades['g'] - distance_mod_obs
data_obs_Pleiades['bp_abs'] = data_obs_Pleiades['bp'] - distance_mod_obs
data_obs_Pleiades['rp_abs'] = data_obs_Pleiades['rp'] - distance_mod_obs
data_obs_Pleiades['Jmag_abs'] = data_obs_Pleiades['Jmag'] - distance_mod_obs
data_obs_Pleiades['Hmag_abs'] = data_obs_Pleiades['Hmag'] - distance_mod_obs
data_obs_Pleiades['Kmag_abs'] = data_obs_Pleiades['Kmag'] - distance_mod_obs
data_obs_Pleiades['rmag_abs'] = data_obs_Pleiades['rmag'] - distance_mod_obs
data_obs_Pleiades['imag_abs'] = data_obs_Pleiades['imag'] - distance_mod_obs
data_obs_Pleiades['ymag_abs'] = data_obs_Pleiades['ymag'] - distance_mod_obs
data_obs_Pleiades['zmag_abs'] = data_obs_Pleiades['zmag'] - distance_mod_obs

### PARSEC

In [5]:
PARSEC_iso_omega_00 = np.loadtxt('/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/data/PARSEC_iso_omega_06.dat')


In [6]:
len(PARSEC_iso_omega_00)

In [7]:
with open('/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/data/PARSEC_iso_omega_06.dat', 'r') as file:
    for i, line in enumerate(file):
        if i == 14:
            columns = line.strip().lstrip('#').split()
            break

In [8]:
PARSEC_iso_omega_00_dataframe = pd.DataFrame(PARSEC_iso_omega_00, columns=columns)


In [9]:
PARSEC_iso_omega_00_dataframe

In [10]:
PARSEC_iso_omega_00_dataframe.rename(columns={'Mass': 'M/Ms'}, inplace=True)

In [11]:
PARSEC_iso_omega_00_dataframe.columns[0:50]

In [12]:
PARSEC_iso_omega_00_dataframe['M/Ms']

In [13]:
PARSEC_iso_omega_00_dataframe['Age [Gyr]'] = (10**(PARSEC_iso_omega_00_dataframe['logAge']))/(1e9)

In [14]:
PARSEC_iso_omega_00_dataframe.to_csv('PARSEC_iso_omega_06.csv', index=False)


In [15]:
lib = ['Gaia', '2MASS', 'PanSTARRS']

PARSEC_iso_omega_00_Phot = {}

for l in lib:
    data = np.loadtxt('/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/data/PARSEC_iso_omega_00_'+l+'.dat')
    
    # Leer la fila 15 para obtener los nombres de las columnas
    with open('/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/data/PARSEC_iso_omega_00_'+l+'.dat', 'r') as file:
        for i, line in enumerate(file):
            if i == 14:
                columns = line.strip().lstrip('#').split()
                break
    
    # Crear un DataFrame con los datos y los nombres de las columnas
    dataframe = pd.DataFrame(data, columns=columns)
    
    dataframe.rename(columns={'Mass': 'M/Ms'}, inplace=True)
    
    # Guardar el DataFrame en el diccionario con el nombre correspondiente
    PARSEC_iso_omega_00_Phot[l] = dataframe
    
    PARSEC_iso_omega_00_Phot[l]['Age [Gyr]'] = (10**(PARSEC_iso_omega_00_Phot[l]['logAge']))/(1e9)
    
common_columns = PARSEC_iso_omega_00_Phot['Gaia'].columns[:35]

dataframes_concatenate = [df for df in PARSEC_iso_omega_00_Phot.values()]

dataframe_final = pd.concat(dataframes_concatenate, axis=1)

dataframe_final = dataframe_final.loc[:,~dataframe_final.columns.duplicated()]

dataframe_final.to_csv('/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/data/PARSEC_iso_omega_00_Phot.csv', index=False)

PARSEC_iso_omega_00_Phot_dict = {}

for age in dataframe_final['Age [Gyr]'].unique():
    dataframe_by_age = dataframe_final[dataframe_final['Age [Gyr]'] == age]
    dataframe_by_age = dataframe_by_age.loc[dataframe_by_age['G_i00'] > 0].reset_index(drop=True)
    PARSEC_iso_omega_00_Phot_dict[age] = dataframe_by_age


In [16]:
dataframe_final['Age [Gyr]']

In [17]:
PARSEC_iso_omega_00_Phot['Gaia'].columns

In [18]:
PARSEC_iso_omega_00_Phot_dict = {round(key, 3): value for key, value in PARSEC_iso_omega_00_Phot_dict.items()}


In [19]:
PARSEC_iso_omega_00_Phot_dict.keys()

### BT-Settl

In [20]:
path_models = '/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/data/BT-Settl_all_Myr_Gaia+2MASS+PanSTARRS.csv'


In [21]:
BTSettl = pd.read_csv(path_models)
BTSettl

In [22]:
BTSettl.rename(columns={
    'age_Myr': 'age_Gyr',
    'Teff(K)': 'Teff',
    'G_RP': 'RP',
    'G_BP': 'BP',
    'r_p1': 'r',
    'i_p1': 'i',
    'y_p1': 'y',
    'z_p1': 'z'}, inplace=True)

BTSettl['age_Gyr'] *= 0.001
BTSettl['A(Li)'] = np.log10(BTSettl['Li']) + 3.3

# Create a dictionary to store dataframes for each isochrone
BTSettl_Li_isochrones = {}

# Loop over each row in the model dataframe
for index, row in BTSettl.iterrows():
    # Get the value of age_Gyr from the current row
    age_Gyr = row['age_Gyr']

    # Check if the value of age_Gyr already exists as a key in the dictionary
    if age_Gyr not in BTSettl_Li_isochrones:
        # If it doesn't exist, create a new entry in the dictionary with the value of age_Gyr as the key
        BTSettl_Li_isochrones[age_Gyr] = []

    # Add the current row to the corresponding value of age_Gyr in the dictionary
    BTSettl_Li_isochrones[age_Gyr].append(row)

# Convert each list of rows into a dataframe and replace the list in the dictionary
for age_Gyr, rows in BTSettl_Li_isochrones.items():
    BTSettl_Li_isochrones[age_Gyr] = pd.DataFrame(rows)



In [23]:
BTSettl_Li_isochrones[0.08]

### MIST

In [24]:
import sys
sys.path.append('/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/')

In [25]:
import read_mist_models

In [26]:
iso_00_00_G2 = read_mist_models.ISOCMD('/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/data/MIST_v1.2_feh_p0.00_afe_p0.0_vvcrit0.0_UBVRIplus.iso.cmd')


In [27]:
print('version: ', iso_00_00_G2.version)
print('abundances: ', iso_00_00_G2.abun)
print('rotation: ', iso_00_00_G2.rot)
print('ages: ', [round(x,2) for x in iso_00_00_G2.ages])
print('number of ages: ', iso_00_00_G2.num_ages)
print('available columns: ', iso_00_00_G2.hdr_list)

In [28]:
iso_00_00_P = read_mist_models.ISOCMD('/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/data/MIST_v1.2_feh_p0.00_afe_p0.0_vvcrit0.0_PanSTARRS.iso.cmd')

In [29]:
print('version: ', iso_00_00_P.version)
print('abundances: ', iso_00_00_P.abun)
print('rotation: ', iso_00_00_P.rot)
print('ages: ', [round(x,2) for x in iso_00_00_P.ages])
print('number of ages: ', iso_00_00_P.num_ages)
print('available columns: ', iso_00_00_P.hdr_list)

In [30]:
def isocmd_to_dataframe(isocmd_set, ages):
    data_dict = {}
    for age, isocmd in zip(ages, isocmd_set):
        data_dict[age] = pd.DataFrame(isocmd)
    return data_dict

filename_iso_P = '/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/data/MIST_v1.2_feh_p0.00_afe_p0.0_vvcrit0.0_PanSTARRS.iso.cmd'
filename_iso_G2 = '/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/data/MIST_v1.2_feh_p0.00_afe_p0.0_vvcrit0.0_UBVRIplus.iso.cmd'

iso_P = read_mist_models.ISOCMD(filename_iso_P)
iso_G2 = read_mist_models.ISOCMD(filename_iso_G2)

ages_P_sorted = sorted(iso_P.ages)
ages_G2_sorted = sorted(iso_G2.ages)

MIST_P = isocmd_to_dataframe(iso_P.isocmds, ages_P_sorted)
MIST_G2 = isocmd_to_dataframe(iso_G2.isocmds, ages_G2_sorted)

MIST_P = {round((10 ** age) / 1e9, 4): df for age, df in MIST_P.items()}
MIST_G2 = {round((10 ** age) / 1e9, 4): df for age, df in MIST_G2.items()}


In [31]:
MIST_P.keys()

In [32]:
MIST_P[0.0794]

In [33]:
MIST_G2[0.0794]

In [34]:
MIST_00_00 = {}

for age in MIST_P.keys():
    if age in MIST_G2:
        df_P = MIST_P[age]
        df_G2 = MIST_G2[age]
        
        combined_df = pd.concat([df_P.iloc[:, :9], df_P.iloc[:, 9:], df_G2.iloc[:, 9:]], axis=1)
        combined_df = combined_df.loc[combined_df['Gaia_G_EDR3'] > 0].reset_index(drop=True)
        MIST_00_00[age] = combined_df


In [35]:
MIST_00_00[0.0794].columns

In [36]:
column_mapping = {
    'star_mass': 'M/Ms',
    'Gaia_G_EDR3': 'G',
    'Gaia_BP_EDR3': 'BP',
    'Gaia_RP_EDR3': 'RP',
    '2MASS_J': 'J',
    '2MASS_H': 'H',
    '2MASS_Ks': 'K',
    'PS_g': 'g',
    'PS_r': 'r',
    'PS_i': 'i',
    'PS_z': 'z',
    'PS_y': 'y',
    'PS_w': 'w'
}

for age, df in MIST_00_00.items():
    MIST_00_00[age] = df.rename(columns=column_mapping)

In [37]:
MIST_00_00[0.0794].columns

In [38]:
iso_00_04_G2 = read_mist_models.ISOCMD('/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/data/MIST_v1.2_feh_p0.00_afe_p0.0_vvcrit0.4_UBVRIplus.iso.cmd')


In [39]:
iso_00_04_P = read_mist_models.ISOCMD('/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/data/MIST_v1.2_feh_p0.00_afe_p0.0_vvcrit0.4_PanSTARRS.iso.cmd')

In [40]:
filename_iso_P = '/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/data/MIST_v1.2_feh_p0.00_afe_p0.0_vvcrit0.4_PanSTARRS.iso.cmd'
filename_iso_G2 = '/pcdisk/dalen/lgonzalez/OneDrive/Escritorio/CAB_INTA-CSIC/01-Doctorado/013-Jupyter/0134-biosc_env/data/MIST_v1.2_feh_p0.00_afe_p0.0_vvcrit0.4_UBVRIplus.iso.cmd'

iso_P = read_mist_models.ISOCMD(filename_iso_P)
iso_G2 = read_mist_models.ISOCMD(filename_iso_G2)

ages_P_sorted = sorted(iso_P.ages)
ages_G2_sorted = sorted(iso_G2.ages)

MIST_P = isocmd_to_dataframe(iso_P.isocmds, ages_P_sorted)
MIST_G2 = isocmd_to_dataframe(iso_G2.isocmds, ages_G2_sorted)

MIST_P = {round((10 ** age) / 1e9, 4): df for age, df in MIST_P.items()}
MIST_G2 = {round((10 ** age) / 1e9, 4): df for age, df in MIST_G2.items()}


In [41]:
MIST_00_04 = {}

for age in MIST_P.keys():
    if age in MIST_G2:
        df_P = MIST_P[age]
        df_G2 = MIST_G2[age]
        
        combined_df = pd.concat([df_P.iloc[:, :9], df_P.iloc[:, 9:], df_G2.iloc[:, 9:]], axis=1)
        combined_df = combined_df.loc[combined_df['Gaia_G_EDR3'] > 0].reset_index(drop=True)
        MIST_00_04[age] = combined_df


In [42]:
for age, df in MIST_00_04.items():
    MIST_00_04[age] = df.rename(columns=column_mapping)

In [43]:
MIST_00_04[0.0794].columns

### MIST $A(Li)$

### Isochrones plots

In [44]:
len(BTSettl_Li_isochrones[0.120]['G'])

In [45]:
def plot_isochrones(model1_dict, model2_dict, band1m1, band2m1, band1m2, band2m2, mod1, mod2):
    # Set the font size
    plt.rcParams.update({'font.size': 26})

    # Create the figure and subplots
    fig, axs = plt.subplots(4, 1, figsize=(10, 28), sharex=True)

    # List of isochrones to use
    isochrones = [0.02, 0.08, 0.12, 0.6]

    # Iterate over each subplot and plot
    for i, ax in enumerate(axs):
        isochrone = isochrones[i]
        ax.plot(model1_dict[isochrone][band1m1] - model1_dict[isochrone][band2m1], model1_dict[isochrone][band1m1], label=f'{mod1}')
        isochrone_parsec = min(model2_dict.keys(), key=lambda x: abs(x - isochrone))
        ax.plot(model2_dict[isochrone_parsec][band1m2] - model2_dict[isochrone_parsec][band2m2], model2_dict[isochrone_parsec][band1m2], label=f'{mod2}')   
        # Set labels and axes adjustments
        if i == 3:  # Only the last subplot has labels and ticks
            ax.set_xlabel(f'{band1m1}-{band2m1} [mag]')
            ax.set_ylabel(f'{band1m1} [mag]')
            if band2m1 == 'RP':
                ax.set_xlim(0, 1.6)
                ax.set_ylim(1, 15.5)
            elif band2m1 == 'J':
                ax.set_xlim(0, 6.5)
                ax.set_ylim(1, 18.75)
            elif band2m1 == 'K':
                ax.set_xlim(0, 2.5)
                ax.set_ylim(1, 18.75)
            elif band2m1 == 'y':
                ax.set_xlim(0, 2.8)
                ax.set_ylim(1, 18.75)
            elif band2m1 == 'z':
                ax.set_xlim(0, 3.25)
                ax.set_ylim(1, 18.75)
            ax.xaxis.set_major_locator(ticker.AutoLocator())
            ax.xaxis.set_major_formatter(ticker.ScalarFormatter())
            ax.legend()
        else:  # Hide labels and ticks for other subplots
            if band2m1 == 'RP':
                ax.set_xlim(0, 1.6)
                ax.set_ylim(1, 15.5)
            elif band2m1 == 'J':
                ax.set_xlim(0, 6.5)
                ax.set_ylim(1, 18.75)
            elif band2m1 == 'K':
                ax.set_xlim(0, 2.5)
                ax.set_ylim(1, 18.75)
            elif band2m1 == 'y':
                ax.set_xlim(0, 2.8)
                ax.set_ylim(1, 18.75)
            elif band2m1 == 'z':
                ax.set_xlim(0, 3.25)
                ax.set_ylim(1, 18.75)
            ax.set_ylabel(f'{band1m1} [mag]')
            ax.set_xticklabels([])
            ax.tick_params(axis='both', which='both', length=5)
            ax.yaxis.get_major_ticks()[0].label1.set_visible(False)
        ax.invert_yaxis()

    # Adjust the spacing between subplots
    plt.subplots_adjust(hspace=0)

    # Show the plot
    return fig, axs

def plot_isochrones_grid(model1_dict, model2_dict, bands1, bands2, mod1, mod2, filename=None, dpi=350, data_obs=None, obs=False):
    # Set the font size
    plt.rcParams.update({'font.size': 26})

    # Create the figure and subplots
    fig, axs = plt.subplots(4, 5, figsize=(55, 28), sharex='col')

    # List of isochrones to use
    isochrones = [0.02, 0.08, 0.12, 0.6]

    # Iterate over each column (band combination)
    for i, (band1, band2) in enumerate(zip(bands1, bands2)):
        # Initialize variables to store the maximum and minimum x-axis values for this column
        max_x_column = float('-inf')
        max_y = float('-inf')

        # Iterate over each row (isochrone)
        for j, isochrone in enumerate(isochrones):
            ax = axs[j, i]
            isochrone_1 = min(model1_dict.keys(), key=lambda x: abs(x - isochrone))
            isochrone_2 = min(model2_dict.keys(), key=lambda x: abs(x - isochrone))

            x1 = model1_dict[isochrone_1][band1[0]] - model1_dict[isochrone_1][band1[1]]
            y1 = model1_dict[isochrone_1][band1[0]]
            x2 = model2_dict[isochrone_2][band2[0]] - model2_dict[isochrone_2][band2[1]]
            y2 = model2_dict[isochrone_2][band2[0]]

            # Plot the isochrones
            ax.plot(x1, y1, label=f'{get_first_word(mod1)}', zorder=2, linewidth=2)
            ax.plot(x2, y2, label=f'{get_first_word(mod2)}', zorder=1, linewidth=2)

            # Find intersection points
            intersection_points = []
            for x1_val, y1_val in zip(x1, y1):
                for x2_val, y2_val in zip(x2, y2):
                    if abs(x1_val - x2_val) < 0.01 and abs(y1_val - y2_val) < 0.01:
                        intersection_points.append((x1_val, y1_val))

            max_x_column = max(max_x_column, np.max(x1), np.max(x2))
            max_y = max(max_y, np.max(y1), np.max(y2))
            
            # Plot the segment connecting the last intersection point to the axes
            if intersection_points:
                x_first, y_first = intersection_points[0]
                if 0 <= x_first <= max_x_column and 1 <= y_first <= max_y:
                    ax.plot([x_first, x_first], [max_y, y_first], color='k', linestyle='--', linewidth=1, zorder=0)
                    ax.plot([-0.2, x_first], [y_first, y_first], color='k', linestyle='--', linewidth=1, zorder=0)
                    ax.text(x_first+(1/20)*(max_x_column+0.2), y_first-(1/20)*(max_y-1), f'({round(x_first,3)}, {round(y_first,2)})')
            
            # Plot observational data if available
            if obs:
                ax.errorbar(data_obs[f'{bandsobs[i][0]}'+'_abs']-data_obs[f'{bandsobs[i][1]}'+'_abs'],
                            data_obs[f'{bandsobs[i][0]}'+'_abs'], 
                            yerr=data_obs['e_'+f'{bandsobs[i][0]}'], fmt='.', label='Obs', zorder=0, color='r')

            # Set labels and axes adjustments
            if j == 3:  # Only the last row has labels and ticks
                ax.set_xlabel(f'{band1[0]}-{band1[1]} [mag]')
                ax.set_ylabel(f'{band1[0]} [mag]')
                ax.xaxis.set_major_locator(ticker.AutoLocator())
                ax.xaxis.set_major_formatter(ticker.ScalarFormatter())
                ax.text(0.05, 0.05, f'{isochrone_1} Gyr', transform=ax.transAxes,
                        fontsize=28, verticalalignment='bottom', horizontalalignment='left')

                if i == len(bands1) - 1:  # Check if this is the last column
                    ax.legend(loc='upper right')  # Add legend only for the last column

            else:  # Hide labels and ticks for other rows
                ax.set_ylabel(f'{band1[0]} [mag]')
                ax.set_xticklabels([])
                ax.tick_params(axis='both', which='both', length=5)
                ax.yaxis.get_major_ticks()[0].label1.set_visible(False)

                ax.text(0.05, 0.05, f'{isochrone_1} Gyr', transform=ax.transAxes,
                        fontsize=28, verticalalignment='bottom', horizontalalignment='left')

                if i == len(bands1) - 1:  # Check if this is the last column
                    ax.legend(loc='upper right')  # Add legend only for the last column

            ax.set_xlim(-0.2, max_x_column)
            ax.set_ylim(1, max_y)
            ax.invert_yaxis()

    # Adjust the spacing between subplots
    plt.subplots_adjust(hspace=0, wspace=0.175)

    fig.suptitle(f'CMD Pleiades; {mod1} vs {mod2}', y=0.92)

    # Save the plot
    if filename:
        save_model_comparison(filename, mod1, mod2, dpi=dpi)

    # Show the plot
    plt.show()

In [46]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path

plt.rcParams.update({'font.size': 10})  # Set the font size

def find_intersection(line1, line2):
    intersection_points = []
    for i in range(len(line1) - 1):
        for j in range(len(line2) - 1):
            path1 = Path([line1[i], line1[i+1]])
            path2 = Path([line2[j], line2[j+1]])
            if path1.intersects_path(path2):
                x_int, y_int = get_intersection(line1[i], line1[i+1], line2[j], line2[j+1])
                intersection_points.append((x_int, y_int))
    return intersection_points

def get_intersection(p1, p2, p3, p4):
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    x4, y4 = p4
    denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    if denom == 0:
        return None, None
    x_int = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denom
    y_int = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denom
    return x_int, y_int

# Datos de ejemplo
x1 = np.linspace(0, 20, 10)
y1 = 2 * x1 + 1

x2 = np.linspace(0, 10, 20)
y2 = (4 * x2 + 2)*np.sin(x2)

# Encontrar la intersección entre las líneas
intersection_points = find_intersection(np.column_stack((x1, y1)), np.column_stack((x2, y2)))

# Plot de las líneas
plt.plot(x1, y1, label='Línea 1')
plt.plot(x2, y2, label='Línea 2')
plt.scatter(x1, y1)
plt.scatter(x2, y2)

# Plot de los puntos de intersección
if intersection_points:
    intersection_points = np.array(intersection_points)
    plt.plot(intersection_points[:, 0], intersection_points[:, 1], 'ro', label='Intersección')

print(intersection_points)
    

plt.legend()
plt.show()

In [47]:
bands1 = [['G', 'RP'], ['G', 'J'], ['J', 'K'], ['G', 'y'], ['G', 'z']]
bands2 = [['G_i00', 'G_RP_i00'], ['G_i00', 'J_i00'], ['J_i00', 'Ks_i00'], ['G_i00', 'yP1_i00'], ['G_i00', 'zP1_i00']]
bandsobs = [['g', 'rp'], ['g', 'Jmag'], ['Jmag', 'Kmag'], ['g', 'ymag'], ['g', 'zmag']]


# Function to get the first word of a string
def get_first_word(string):
    return string.split()[0]

# Function to save the model comparison
def save_model_comparison(filename, mod1, mod2, dpi=400):
    mod1_first_word = get_first_word(mod1)
    mod2_first_word = get_first_word(mod2)
    final_filename = f"{filename}_{mod1_first_word}_{mod2_first_word}.png"
    plt.savefig(final_filename, dpi=dpi, bbox_inches='tight')

# Function to find intersection between two lines
def find_intersection(line1, line2):
    intersection_points = []
    for i in range(len(line1) - 1):
        for j in range(len(line2) - 1):
            path1 = Path([line1[i], line1[i+1]])
            path2 = Path([line2[j], line2[j+1]])
            if path1.intersects_path(path2):
                x_int, y_int = get_intersection(line1[i], line1[i+1], line2[j], line2[j+1])
                intersection_points.append((x_int, y_int))
    return intersection_points

# Function to find the intersection point between two line segments
def get_intersection(p1, p2, p3, p4):
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    x4, y4 = p4
    denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    if denom == 0:
        return None, None
    x_int = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denom
    y_int = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denom
    return x_int, y_int    

# Definition of bands
bands1 = [['G', 'RP'], ['G', 'J'], ['J', 'K'], ['G', 'y'], ['G', 'z']]
bands2 = [['G_i00', 'G_RP_i00'], ['G_i00', 'J_i00'], ['J_i00', 'Ks_i00'], ['G_i00', 'yP1_i00'], ['G_i00', 'zP1_i00']]
bandsobs = [['g', 'rp'], ['g', 'Jmag'], ['Jmag', 'Kmag'], ['g', 'ymag'], ['g', 'zmag']]

# Main function to plot the isochrones grid
def plot_isochrones_grid(model1_dict, model2_dict, bands1, bands2, mod1, mod2, filename=None, dpi=350, data_obs=None, obs=False):
    plt.rcParams.update({'font.size': 26})  # Set the font size

    fig, axs = plt.subplots(4, 5, figsize=(55, 28), sharex='col')  # Create the figure and subplots
    isochrones = [0.02, 0.08, 0.12, 0.6]  # List of isochrones to use
    max_mass_labels = 5  # Maximum number of mass labels to show
    
    legend_labels = {f'{get_first_word(mod1)}': f'{get_first_word(mod1)}',
                     f'{get_first_word(mod2)}': f'{get_first_word(mod2)}',
                     'Obs': 'Obs'}
    
    # Iterate over each column (band combination)
    for i, (band1, band2) in enumerate(zip(bands1, bands2)):
        max_x_column = float('-inf')
        max_y = float('-inf')

        # Iterate over each row (isochrone)
        for j, isochrone in enumerate(isochrones):
            ax = axs[j, i]
            isochrone_1 = min(model1_dict.keys(), key=lambda x: abs(x - isochrone))
            isochrone_2 = min(model2_dict.keys(), key=lambda x: abs(x - isochrone))

            model1_df = model1_dict[isochrone_1]
            model1_df = model1_df.loc[model1_df['M/Ms'] < 1.5]
            model1_df = model1_df.loc[model1_df['M/Ms'] > 0.08]
            
            model2_df = model2_dict[isochrone_2]
            model2_df = model2_df.loc[model2_df['M/Ms'] < 1.5]
            model2_df = model2_df.loc[model2_df['M/Ms'] > 0.08]
            
            min_mm_model1 = min(model1_df['M/Ms'])
            min_mm_model2 = min(model2_df['M/Ms'])

            mm_values = np.linspace(1.5, min(min_mm_model1, min_mm_model2), 5)

            x1 = model1_dict[isochrone_1][band1[0]] - model1_dict[isochrone_1][band1[1]]
            y1 = model1_dict[isochrone_1][band1[0]]
            x2 = model2_dict[isochrone_2][band2[0]] - model2_dict[isochrone_2][band2[1]]
            y2 = model2_dict[isochrone_2][band2[0]]

            ax.plot(x1, y1, label=f'{get_first_word(mod1)}', zorder=2, linewidth=2, color='blue')
            ax.plot(x2, y2, label=f'{get_first_word(mod2)}', zorder=1, linewidth=2, color='orange')

            max_x_column = max(max_x_column, np.max(x1), np.max(x2))
            max_y = max(max_y, np.max(y1), np.max(y2))
            
            for mm_value in mm_values:
                closest_row_model1 = model1_df.iloc[(model1_df['M/Ms'] - mm_value).abs().argsort()[:1]]
                closest_row_model2 = model2_df.iloc[(model2_df['M/Ms'] - mm_value).abs().argsort()[:1]]

                x_mass_model1 = closest_row_model1[band1[0]] - closest_row_model1[band1[1]]
                y_mass_model1 = closest_row_model1[band1[0]]
                mass_model1 = closest_row_model1['M/Ms']

                x_mass_model2 = closest_row_model2[band2[0]] - closest_row_model2[band2[1]]
                y_mass_model2 = closest_row_model2[band2[0]]
                mass_model2 = closest_row_model2['M/Ms']

                for x_mass1, y_mass1, mass1, x_mass2, y_mass2, mass2 in zip(x_mass_model1, y_mass_model1, mass_model1, x_mass_model2, y_mass_model2, mass_model2):
                    text = f'{((mass1+mass2)/2):.3f}'
                    text_length = len(text) * 0.15  # Estimar la longitud del texto (suponiendo 0.15 como el ancho aproximado por carácter)
                    text_height = 0.03  # Estimar la altura del texto (suponiendo 0.03)

                    if x_mass2 - x_mass1 < 0:
                        text_x = x_mass1 + (1/30)*(max_x_column+0.2)
                        ax.text(text_x, y_mass1, text, fontsize=12, color='k')
                    elif x_mass1 - x_mass2 < 0:
                        text_x = x_mass2 + (1/30)*(max_x_column+0.2)
                        ax.text(text_x, y_mass2, text, fontsize=12, color='k')

                    ax.plot([x_mass1, x_mass2], [y_mass1, y_mass2], color='k', linewidth=1)

            intersection_points = find_intersection(np.column_stack((x1, y1)), np.column_stack((x2, y2)))

            if intersection_points:
                first_point = intersection_points[-1]
                if -0.2 <= first_point[0] <= max_x_column and 1 <= first_point[1] <= max_y:
                    ax.plot([-0.2, first_point[0]], [first_point[1], first_point[1]], color='black', linewidth=1, linestyle='--')
                    ax.plot([first_point[0], first_point[0]], [max_y, first_point[1]], color='black', linewidth=1, linestyle='--')
                    ax.text(0.55, 0.95, f'({first_point[0]:.2f}, {first_point[1]:.2f})', fontsize=18, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, color='black')  # Adjusted position for text
                else:
                    pass
            
            # Plot observational data if available
            if obs:
                ax.errorbar(data_obs[f'{bandsobs[i][0]}'+'_abs']-data_obs[f'{bandsobs[i][1]}'+'_abs'],
                            data_obs[f'{bandsobs[i][0]}'+'_abs'], 
                            yerr=data_obs['e_'+f'{bandsobs[i][0]}'], fmt='.', alpha=0.125, label='Obs', zorder=0, color='r')

            if j == 3:
                ax.set_xlabel(f'{band1[0]}-{band1[1]} [mag]')
                ax.set_ylabel(f'{band1[0]} [mag]')
                ax.xaxis.set_major_locator(ticker.AutoLocator())
                ax.xaxis.set_major_formatter(ticker.ScalarFormatter())
                ax.text(0.05, 0.05, f'{isochrone_1} Gyr', transform=ax.transAxes, fontsize=28, verticalalignment='bottom', horizontalalignment='left')  # Adjusted position for text

            else:
                ax.set_ylabel(f'{band1[0]} [mag]')
                ax.set_xticklabels([])
                ax.tick_params(axis='both', which='both', length=5)

                ax.text(0.05, 0.05, f'{isochrone_1} Gyr', transform=ax.transAxes, fontsize=28, verticalalignment='bottom', horizontalalignment='left')  # Adjusted position for text

            ax.set_xlim(-0.2, max_x_column+(1/8)*(max_x_column+0.2))
            ax.set_ylim(1, max_y)
            ax.invert_yaxis()

    plt.subplots_adjust(hspace=0, wspace=0.175)
    fig.suptitle(f'CMD Pleiades; {mod1} vs {mod2}', y=0.94)
    handles, labels = ax.get_legend_handles_labels()
    unique_labels = list(set(labels))
    legend_handles = [handles[labels.index(label)] for label in unique_labels]
    fig.legend(legend_handles, unique_labels, loc='upper center', bbox_to_anchor=(0.5, 0.9225), fontsize=24, ncol=3, edgecolor='black')  # Añadir leyenda debajo del título
    if filename:
        save_model_comparison(filename, mod1, mod2, dpi=dpi)
    plt.show()


# Example usage
plot_isochrones_grid(BTSettl_Li_isochrones, PARSEC_iso_omega_00_Phot_dict, bands1, bands2, 'BT-Settl $\odot$', r'PARSEC $\omega_i=0.0$; $Z=0.001547$', filename='model_comparison', data_obs=data_obs_Pleiades, obs=True)

In [48]:
plot_isochrones_grid(BTSettl_Li_isochrones, MIST_00_00, bands1, bands1, 'BT-Settl $\odot$', r'MIST $v/v_{\rm crit}=0.0$; [Fe/H]$=0.0$', filename='model_comparison', data_obs=data_obs_Pleiades, obs=True)


In [49]:
plot_isochrones_grid(MIST_00_00, PARSEC_iso_omega_00_Phot_dict, bands1, bands2, r'MIST $v/v_{\rm crit}=0.0$; [Fe/H]$=0.0$', r'PARSEC $\omega_i=0.0$; $Z=0.001547$', filename='model_comparison', data_obs=data_obs_Pleiades, obs=True)


In [50]:
(1e7*(0.3)**(-2.5))/1e9